In [33]:
import numpy as np
import xgboost as xgb
from skopt import BayesSearchCV
from skopt.space import Integer
from skopt.plots import plot_objective, plot_convergence
import matplotlib.pyplot as plt
from preprocess import get_data
import joblib


In [ ]:
trainData =get_data("train/","ascii_file_counts.csv")
testData = get_data("test","ascii_file_counts.csv")
XTrain, ytrain = trainData
XTest, ytest = testData

In [29]:
model = xgb.XGBClassifier(objective='multi:softmax',n_estimators = 140, max_depth = 8,num_class=93,booster='gbtree',eval_metric= 'mlogloss', random_state=42)

In [24]:
param_space = {
    'n_estimators': Integer(10, 200),        # Number of trees
    'max_depth': Integer(3, 15),             # Depth of each tree     
}

In [25]:
bayes_opt = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,
    n_iter=10,  # Number of iterations
    cv=5,       # 5-fold cross-validation
    scoring='neg_mean_squared_error',  # Objective: minimize error
    n_jobs=-1,  # Use all CPU cores
    random_state=42,
    verbose=3
)

In [30]:
bayes_opt.fit(XTrain, ytrain)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


python(21119) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(21120) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(21121) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(21122) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(21123) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(21124) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(21125) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(21126) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


KeyboardInterrupt: 

In [31]:
model.fit(XTrain, ytrain)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=140,
              n_jobs=None, num_class=93, num_parallel_tree=None, ...)

In [32]:
from testing_models import evaluate_model
y_pred = model.predict(XTest)
print(evaluate_model(ytest, y_pred))

(0.7424123995685852, 0.7451540126156821, array([[  28,    0,    0, ...,    0,    0,    0],
       [   0,   46,    0, ...,    0,    0,    0],
       [   0,    0,    7, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,    3,    0,    0],
       [   1,    0,    0, ...,    0,    2,    0],
       [   0,    0,    0, ...,    0,    0, 1002]]), 0.7455758956660402, 0.7451540126156821)


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [35]:
joblib.dump(model, 'xgboost.joblib')

['xgboost.joblib']